# Animal Counting using Traditional Image Processing

This notebook demonstrates how to count animals in images using basic image processing techniques without deep learning models.

In [1]:
# Import necessary libraries
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage
import os

In [2]:
images_counting = []
images_finding = []

# Function to read all images from a folder
def read_images_from_folder(folder_path):
    images = []
    for filename in os.listdir(folder_path):
        # Check if the file is an image (you can add more extensions if needed)
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            img_path = os.path.join(folder_path, filename)
            print(img_path)
            img = cv2.imread(img_path)
            if img is not None:
                # Convert from BGR to RGB for display purposes
                img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                images.append(img_rgb)
                print(f"Loaded: {filename}")
            else:
                print(f"Failed to load: {filename}")
    return images

# Define paths to the Counting and Finding folders
counting_folder_path = "Counting"
finding_folder_path = "Finding"

# Load images from both folders
print("Loading images from Counting folder...")
images_counting = read_images_from_folder(counting_folder_path)
print(f"Loaded {len(images_counting)} images from Counting folder.\n")

print("Loading images from Finding folder...")
images_finding = read_images_from_folder(finding_folder_path)
print(f"Loaded {len(images_finding)} images from Finding folder.")

# Display a sample image from each folder if available
plt.figure(figsize=(12, 6))

if len(images_counting) > 0:
    plt.subplot(1, 2, 1)
    plt.imshow(images_counting[0])
    plt.title("Sample from Counting folder")
    plt.axis('off')

if len(images_finding) > 0:
    plt.subplot(1, 2, 2)
    plt.imshow(images_finding[0])
    plt.title("Sample from Finding folder")
    plt.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def count_cats(image):
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    
    
    # Apply histogram equalization to improve contrast
    equalized = cv2.equalizeHist(gray)
          
    
    # Apply Gaussian blur to reduce noise
    blur = cv2.GaussianBlur(equalized, (5, 5), 0)
    
    # Apply adaptive thresholding
    thresh = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                   cv2.THRESH_BINARY_INV, 11, 2)
    
    # Morphological operations to clean up the image
    kernel = np.ones((3, 3), np.uint8)
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=2)
    
    # Find contours
    contours, _ = cv2.findContours(opening, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Filter contours by size to eliminate noise
    min_contour_area = 500  # Adjust based on your images
    filtered_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > min_contour_area]
    
    # Draw bounding boxes around detected cats
    result = image.copy()
    for i, contour in enumerate(filtered_contours):
        x, y, w, h = cv2.boundingRect(contour)
        # Draw rectangle and label
        cv2.rectangle(result, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(result, f'Cat {i+1}', (x, y-10), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    
    # Count the number of filtered contours
    count = len(filtered_contours)
    
    return result, count

# Apply the function to count cats in the first image
if len(images_counting) > 0:
    result_img, number_of_cats = count_cats(images_counting[0])
    
    # Display the original and processed images side by side
    plt.figure(figsize=(12, 6))
    
    plt.subplot(1, 2, 1)
    plt.imshow(images_counting[0])
    plt.title("Original Image")
    plt.axis('off')
    
    plt.subplot(1, 2, 2)
    plt.imshow(result_img)
    plt.title(f"Detected Cats: {number_of_cats}")
    plt.axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print(f"Number of cats detected in the image: {number_of_cats}")
else:
    print("No images found in the counting folder.")
